In [59]:
import sys
import glob
import numpy as np
import random
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import keras
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [36]:
# Data can be downloaded at http://pannous.net/spoken_numbers.tar

In [17]:
SEED = 2017
DATA_DIR = 'processed_data/'
print(DATA_DIR)

processed_data/


In [25]:
files = glob.glob(DATA_DIR + "/*/*.wav")
print(type(files[0]))
print(files[0])
X_train, X_val = train_test_split(files, test_size=0.2, random_state=SEED)

print('# Training examples: {}'.format(len(X_train)))
print('# Validation examples: {}'.format(len(X_val)))

<class 'str'>
processed_data\Daniel\Daniel0.wav
# Training examples: 240
# Validation examples: 60


In [26]:
print(labels)

['Jakub', 'Daniel', 'LudvigPartybox', 'Richard']


In [27]:
labels = []
for i in range(len(X_train)):
    label = X_train[i].split('\\')[1]
    if label not in labels:
        labels.append(label)
print(len(labels))

6


In [28]:
label_binarizer = LabelBinarizer()
label_binarizer.fit(list(set(labels)))

def one_hot_encode(x): return label_binarizer.transform(x)

In [29]:
n_features = 20
max_length = 80
n_classes = len(labels)

In [53]:
def batch_generator(data, batch_size=16):
    while 1:
        random.shuffle(data)
        X, y = [], []
        for i in range(batch_size):
            wav = data[i]
            wave, sr = librosa.load(wav, mono=True)
            label = wav.split('\\')[1]
            y.append(label)
            mfcc = librosa.feature.mfcc(wave, sr)
            #mfcc = np.pad(mfcc, ((0,0), (0, max_length-len(mfcc[0]))), mode='constant', constant_values=0) 
            X.append(np.array(mfcc))
        yield np.array(X), np.array(one_hot_encode(y))

In [44]:
learning_rate = 0.001
batch_size = 64
n_epochs = 50
dropout = 0.5

input_shape = (n_features, max_length)
steps_per_epoch = 50

In [45]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=input_shape,
dropout=dropout))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(n_classes, activation='softmax'))

In [46]:
opt = Adam(lr=learning_rate)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 20, 256)           345088    
_________________________________________________________________
flatten_3 (Flatten)          (None, 5120)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               655488    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 774       
Total params: 1,001,350
Trainable params: 1,001,350
Non-trainable params: 0
_________________________________________________________________


In [62]:
callbacks = [ModelCheckpoint('checkpoints/voice_recognition_best_model_{epoch:02d}.hdf5', save_best_only=True),
            EarlyStopping(monitor='val_loss', patience=2)]
#(callbacks[1])

In [63]:
history = model.fit_generator(
 generator=batch_generator(X_train, batch_size),
 steps_per_epoch=steps_per_epoch,
 epochs=n_epochs,
 verbose=1,
 validation_data=batch_generator(X_val, 16),
 validation_steps=5,
 callbacks=callbacks
 )

Epoch 1/50


StopIteration: 

In [54]:
history

In [ ]:
model.save